## RL Hacking 3

Hack in this direction as long as it makes sense: 

- [ ]  Setup a basic LLM (maybe QWEN 0.6B)
- [ ]  A math dataset
- [ ]  Run various policy gradient methods to teach the model to reason (GRPO, Raschka’s simpler GRPO, Dr. GRPO, PPO, Vanilla Policy gradient, maybe DPO, maybe RPOO)
- [ ]  How do implementations perform and vary? Do components make sense? Can we replicated some version of the “aha moment”? (although that’s apparently not an emergent RL property?) Does the way the math is setup make sense? does Karpathy’s simplified explanation make it easier? Would demoing on or more of these algorithms on in a game setting make more sense?

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedModel

In [6]:
# MODEL_NAME = "Qwen/Qwen2.5-3B"
MODEL_NAME = "Qwen/qwen3-0.6B-base"

In [7]:
policy_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map=0,
)
# reference_model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     device_map=0,
# )

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

- Hmm yeah this is like a non-trivial amount of tooling with either repo
- Raschka's seems friendlier, but more complex than I would want in some areas for sure.
- Can I achieve my hacking goals here using Raschka's code?
- Seems like it comes down to if I can make sense of the policy gradient part of his code and modify it. 